[View in Colaboratory](https://colab.research.google.com/github/Naritoshi/colabo_deeplearning_practice/blob/master/Mecab_colab.ipynb)

#Google Colaboratory で、NEologd 辞書 で、MeCab を 使う（Python）


以下の環境でやりました。


*   GoogleColaboratory
*   Python3のノートブック
*   GPUのやつ

##MeCabのインストール

###MeCabをインストールする
[参考にした記事](https://medium.com/moonshot/google-colab-mecab-ipadic-neologd-5e740a57ba73)

MeCabをインストールします。

In [0]:
!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab

mecabをコマンドラインから実行してみます。

In [2]:
%%bash
mecab 
メカブって神ってる

メカブ	名詞,固有名詞,組織,*,*,*,*
って	助詞,格助詞,連語,*,*,*,って,ッテ,ッテ
神	名詞,一般,*,*,*,*,神,カミ,カミ
って	助詞,格助詞,連語,*,*,*,って,ッテ,ッテ
る	助動詞,*,*,*,文語・ル,基本形,る,ル,ル
EOS


「神ってる」が、「神」「って」「る」とわかれてしまします。
なので、新語に対応した辞書「mecab-ipadic-NEologd」を使うようにします。

###mecab-ipadic-NEologdをダウンロード
[mecab-ipadic-NEologdについて](https://engineering.linecorp.com/ja/blog/detail/109)

[参考にした記事](https://qiita.com/bokotomo/items/3e9d78a4bc94c0314346)

mecab-ipadic-NEologdをダウンロードします。

In [0]:
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n

###辞書が新しくなったか確認する１

mecabをコマンドラインから実行してみます。

In [4]:
%%bash
mecab 
メカブって神ってる

メカブ	名詞,固有名詞,組織,*,*,*,*
って	助詞,格助詞,連語,*,*,*,って,ッテ,ッテ
神	名詞,一般,*,*,*,*,神,カミ,カミ
って	助詞,格助詞,連語,*,*,*,って,ッテ,ッテ
る	助動詞,*,*,*,文語・ル,基本形,る,ル,ル
EOS


前回と変わらないようなので、辞書は新しくなっていません。
  
  設定ファイル？「mecabrc」を変更し、新しい辞書のパスを見るようにする必要があるようです。

###設定ファイルを変更する

「/etc/mecabrc」ファイルの「dicdir = /var/lib/mecab/dic/debian」を、
「mecab-ipadic-NEologd」の辞書のパス「/usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd」に変更します。

####/etc/mecabrcの中身を確認する
catコマンドを使って、ファイルの中身を確認します。

In [5]:
!cat /etc/mecabrc 

;
; Configuration file of MeCab
;
; $Id: mecabrc.in,v 1.3 2006/05/29 15:36:08 taku-ku Exp $;
;
dicdir = /var/lib/mecab/dic/debian

; userdic = /home/foo/bar/user.dic

; output-format-type = wakati
; input-buffer-size = 8192

; node-format = %m\n
; bos-format = %S\n
; eos-format = EOS\n



####設定ファイルを変更する

1.   sedコマンドを使て、ファイルを変更、ファイル名を「mecabrc.new」として保存する。
2.   「mecabrc.new」の中身を確認
3.   「mecabrc」のバックアップの保存（mecabrc.orgとする）
4.   「mecabrc.new」を「mecabrc」に上書き
5.   「mecabrc」の中身を確認

ファイルの「dicdir = /var/lib/mecab/dic/debian」を、「mecab-ipadic-NEologd」の辞書のパス「/usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd」に変更します。

In [0]:
!sed -e "s!/var/lib/mecab/dic/debian!/usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd!g" /etc/mecabrc > /etc/mecabrc.new

変わったか確認します。

In [7]:
!cat /etc/mecabrc.new

;
; Configuration file of MeCab
;
; $Id: mecabrc.in,v 1.3 2006/05/29 15:36:08 taku-ku Exp $;
;
dicdir = /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd

; userdic = /home/foo/bar/user.dic

; output-format-type = wakati
; input-buffer-size = 8192

; node-format = %m\n
; bos-format = %S\n
; eos-format = EOS\n



変更出来ているので、「/etc/mecabrc.new」を「/etc/mecabrc」にします。



In [0]:
!cp /etc/mecabrc /etc/mecabrc.org
!cp /etc/mecabrc.new /etc/mecabrc

確認します。

In [9]:
!cat /etc/mecabrc 

;
; Configuration file of MeCab
;
; $Id: mecabrc.in,v 1.3 2006/05/29 15:36:08 taku-ku Exp $;
;
dicdir = /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd

; userdic = /home/foo/bar/user.dic

; output-format-type = wakati
; input-buffer-size = 8192

; node-format = %m\n
; bos-format = %S\n
; eos-format = EOS\n



###辞書が新しくなったか確認する２

In [10]:
%%bash
mecab 
メカブって神ってる

メカブ	名詞,固有名詞,一般,*,*,*,MeCab,メカブ,メカブ
って	助詞,格助詞,連語,*,*,*,って,ッテ,ッテ
神ってる	名詞,固有名詞,一般,*,*,*,神ってる,カミッテル,カミッテル
EOS


##PYTHONで実行

mecab-python3をインストールします。

In [11]:
!pip install mecab-python3

    100% |████████████████████████████████| 51kB 4.3MB/s 
  Running setup.py bdist_wheel for mecab-python3 ... - \ | / - done
  Stored in directory: /content/.cache/pip/wheels/4c/07/3a/5f22ccc9f381f3bc01fa023202061cd1e0e9af855292f005dd
Successfully built mecab-python3


Pythonで、MeCabを使います。

In [12]:
import MeCab
mecab = MeCab.Tagger('-Ochasen')
res = mecab.parse('メカブって神ってる')
print(res)

メカブ	メカブ	MeCab	名詞-固有名詞-一般		
って	ッテ	って	助詞-格助詞-連語		
神ってる	カミッテル	神ってる	名詞-固有名詞-一般		
EOS

